In [1]:
## LIBRARIES
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objs as go

In [2]:
## DATA FROM WEBSITE
first_url = "https://www.olx.pl/nieruchomosci/mieszkania/wynajem/tczew/?search%5Border%5D=created_at%3Adesc&view=list" #first URL sorted by "created at" DESC
first_page = requests.get(first_url)
first_soup = BeautifulSoup(first_page.content, 'html.parser')


## URLS FROM FIRST PAGE
urls = []

for x in first_soup.find_all('a', {'class': 'css-rc5s2u'}):
    if x.get('href').startswith('/d/'):
        urls.append('www.olx.pl' + str(x.get('href')))
    else:
        pass


## MAX EXISTING PAGE FOR SEARCHED CITY
page_list = first_soup.find('ul', {'data-testid': 'pagination-list'})
page_list_chld = page_list.findChildren('li', recursive = False)

pages_number = []
for child in page_list_chld:
    pages_number.append(child.get_text())


## OTHER URLS FROM THE NEXT PAGES
for i in range(int(pages_number[1]),int(pages_number[-1])+1):
    if i <= int(pages_number[-1]):
        other_pages = F"https://www.olx.pl/nieruchomosci/mieszkania/wynajem/tczew/?page={i}&search%5Border%5D=created_at%3Adesc&view=list" #second URL sorted by "created at" DESC with existing number of pages as format
        other_pages_request = requests.get(other_pages)
        other_pages_soup = BeautifulSoup(other_pages_request.content, 'html.parser')
        for x in other_pages_soup.find_all('a', {'class': 'css-rc5s2u'}):
            if x.get('href').startswith('/d/'):
                urls.append('www.olx.pl' + str(x.get('href')))
            else:
                pass
    else:
        pass

In [7]:
## IMPORTING BUILDED FUNCTIONS
import import_ipynb
from functions import *


## FINAL DATA FROM WEBSITES
final_data = []


for link in urls:
    page_url = F'https://{link}'
    page_request = requests.get(page_url)
    page_soup = BeautifulSoup(page_request.content, 'html.parser')
    if 'Tczew' in (page_soup.find_all('a', {'class': 'css-tyi2d1'})[5].get_text().split()):
        final_data.append({
            "URL": link,
            "TITLE": str(page_soup.find('h4', {'class': 'css-1juynto'}).get_text()),
            "ADD_DATE": into_date(page_soup.find('span', {'class': 'css-19yf5ek'}).get_text()),
            "PRICE": float(page_soup.find('h3', {'class': 'css-12vqlj3'}).get_text().replace('zł', '').replace(' ', '')),
            "ADDITIONAL_RENT": float(additional_rent(page_soup.find_all('p', {'class': 'css-b5m1rv er34gjf0'}))),
            "TOTAL_PRICE": float(page_soup.find('h3', {'class': 'css-12vqlj3'}).get_text().replace('zł', '').replace(' ', '')) + 
            float(additional_rent(page_soup.find_all('p', {'class': 'css-b5m1rv er34gjf0'}))),
            "AREA": float(null_insert(area(page_soup.find_all('p', {'class': 'css-b5m1rv er34gjf0'})),-99)),
            "FLOOR": int(null_insert(floor_nr(page_soup.find_all('p', {'class': 'css-b5m1rv er34gjf0'})),-99)),
            "ROOMS": int(null_insert(rooms(page_soup.find_all('p', {'class': 'css-b5m1rv er34gjf0'})),-99)),
            "HAS_BALCONY": has_balcony(page_soup.find('div', {'class': 'css-1t507yq er34gjf0'}).get_text()),
        })
    else:
        pass

In [8]:
##CONVERT DATA INTO DF AND INTO SQL FILE
df = pd.DataFrame(final_data)
df = df.replace(-999, np.NaN) ##NULL VALUES


engine = create_engine('sqlite:///mieszkania.db', echo=False)
df.to_sql('mieszkania', con=engine, if_exists='replace', index=False)

42

In [9]:
## CHART EXAMPLE
df['ROOMS'] = df['ROOMS'].astype(str)

fig = px.scatter(df, x = "PRICE", y = "AREA", color = "ROOMS",
                 labels = {
                     "AREA": "Area (m2)",
                     "PRICE": "Price (zl)",
                     "ROOMS": "No. of rooms"
                 },
                 title ="Comparison of prices with the size of rented apartments in Tczew (Poland, Pomorskie)"
)

fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()